In [1]:
import random
import cv2
import os
import glob
from IPython.display import Image, display



# Set the directory containing the images
image_directory = 'LPs'
# Set the maximum allowed aspect ratio (width/height or height/width)
max_aspect_ratio = 2.5  # for example, this will remove images that are more than 3 times as wide as they are tall or vice versa

def remove_rectangular_images(directory, max_aspect_ratio):
    # Find all image files in the directory and subdirectories
    image_paths = glob.glob(os.path.join(directory, '**', '*'), recursive=True)
    image_paths = [path for path in image_paths if path.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for image_path in image_paths:
        # Read the image using OpenCV
        image = cv2.imread(image_path)
        if image is not None:
            height, width = image.shape[:2]

            # Calculate the aspect ratio
            aspect_ratio = max(width / height, height / width)

            # Check if the aspect ratio exceeds the maximum allowed aspect ratio
            if aspect_ratio > max_aspect_ratio:
                # move the image to a new directory LPs_wide
                os.rename(image_path, os.path.join('LPs_wide', os.path.basename(image_path)))                
                # display the image using iPython display
                # display(Image(image_path))
                # print(f'Removed: {image_path} - Aspect Ratio: {aspect_ratio}')
        else:
            print(f'Error reading: {image_path}. It might not be an image file.')

if __name__ == '__main__':
    remove_rectangular_images(image_directory, max_aspect_ratio)


In [7]:
import cv2
import numpy as np
import os
import random

# Set your directories here
license_plates_dir = 'LPs'  # Path to the directory containing license plate images
backgrounds_dir = 'backgrounds_without_LPs'        # Path to the directory containing background images
image_train_output_dir = 'LPs_superimposed/train/images'                  # Path to the directory where you want to save the superimposed images
image_valid_output_dir = 'LPs_superimposed/valid/images'                  # Path to the directory where you want to save the superimposed images
annotations_train_output_dir = 'LPs_superimposed/train/labels'        # Path to the directory where you want to save the annotations
annotations_valid_output_dir = 'LPs_superimposed/valid/labels'        # Path to the directory where you want to save the annotations

# Create the output directories if they don't exist
os.makedirs(image_train_output_dir, exist_ok=True)
os.makedirs(annotations_train_output_dir, exist_ok=True)
os.makedirs(image_valid_output_dir, exist_ok=True)
os.makedirs(annotations_valid_output_dir, exist_ok=True)

def superimpose_license_plate(background_img, plate_img):
    # Get dimensions of the background and plate images
    bg_height, bg_width, _ = background_img.shape
    plate_height, plate_width, _ = plate_img.shape
    
    # Check if the plate image needs to be resized
    if plate_width > bg_width or plate_height > bg_height:
        # Calculate the scale factor
        scale_width = bg_width / plate_width
        scale_height = bg_height / plate_height
        scale_factor = min(scale_width, scale_height)
        
        # Resize the plate image
        new_width = int(plate_width * scale_factor)
        new_height = int(plate_height * scale_factor)
        plate_img = cv2.resize(plate_img, (new_width, new_height), interpolation=cv2.INTER_AREA)
        plate_height, plate_width = new_height, new_width

    # Now we're sure that plate image will fit within the background image
    max_x = bg_width - plate_width
    max_y = bg_height - plate_height

    x_offset = random.randint(0, max_x) if max_x > 0 else 0
    y_offset = random.randint(0, max_y) if max_y > 0 else 0

    # Ensure the placement does not exceed background dimensions
    end_x = x_offset + plate_width
    end_y = y_offset + plate_height
    
    # Superimpose the plate onto the background
    background_img[y_offset:end_y, x_offset:end_x] = plate_img

    # Return the superimposed image and the bounding box coordinates
    bbox = (x_offset, y_offset, end_x, end_y)
    return background_img, bbox


def save_annotations(file_path, bbox, img_width, img_height):
    # Normalize the bounding box coordinates
    x_center = (bbox[0] + bbox[2]) / 2 / img_width
    y_center = (bbox[1] + bbox[3]) / 2 / img_height
    width = (bbox[2] - bbox[0]) / img_width
    height = (bbox[3] - bbox[1]) / img_height

    # Save the class identifier and normalized bounding box coordinates
    with open(file_path, 'w') as f:
        f.write(f'0 {x_center:.5f} {y_center:.5f} {width:.5f} {height:.5f}\n')


def main():
    # Get all the file names
    license_plates_files = [f for f in os.listdir(license_plates_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    background_files = [f for f in os.listdir(backgrounds_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    for plate_file in license_plates_files:
        # Read the license plate image
        plate_img = cv2.imread(os.path.join(license_plates_dir, plate_file))
        
        # Check if the image is valid
        if plate_img is None:
            print(f"Error reading {plate_file}. Skipping.")
            continue
        
        plate_img = cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        
        # Randomly select a background image and read it
        bg_file = random.choice(background_files)
        background_img = cv2.imread(os.path.join(backgrounds_dir, bg_file))
        
        # Check if the background is valid
        if background_img is None:
            print(f"Error reading {bg_file}. Skipping.")
            continue
        
        background_img = cv2.cvtColor(background_img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        
        # Superimpose the license plate onto the background
        combined_img, bbox = superimpose_license_plate(background_img, plate_img)
        
        images_output_dir = image_train_output_dir
        annotations_output_dir = annotations_train_output_dir
        if random.random() < 0.2:
            annotations_output_dir = annotations_valid_output_dir
            images_output_dir = image_valid_output_dir
        # Save the superimposed image
        output_file_path = os.path.join(images_output_dir, f'superimposed_{plate_file}')
        cv2.imwrite(output_file_path, cv2.cvtColor(combined_img, cv2.COLOR_RGB2BGR))  # Convert back to BGR for saving
        
        # Get background dimensions
        bg_height, bg_width, _ = background_img.shape
        
        # Save the annotations with normalized coordinates
        annotation_file_path = os.path.join(annotations_output_dir, f'superimposed_{plate_file.split()[0]}.txt')
        save_annotations(annotation_file_path, bbox, bg_width, bg_height)

        # print(f'Processed {plate_file}, background {bg_file}, saved to {output_file_path}, bbox: {bbox}')

if __name__ == '__main__':
    main()


In [8]:
# copy those directories to the dataset directory
os.system('cp LPs_superimposed/train/images/* dataset/train/images/')
os.system('cp LPs_superimposed/valid/images/* dataset/valid/images/')
os.system('cp LPs_superimposed/train/labels/* dataset/train/labels/')
os.system('cp LPs_superimposed/valid/labels/* dataset/valid/labels/')

0